In [1]:
# 항상 첫 번째 셀
import sys
import os
sys.path.append(os.path.abspath("../src"))

In [2]:
from db_client import RDSClient

def main():
    # 1. DB 클라이언트 인스턴스 생성 (이때 풀이 생성됨)
    db = RDSClient()
    
    # 2. 쿼리 실행 예시 (SELECT)
    print("--- 사용자 조회 ---")
    users = db.execute("SELECT * FROM crawl_target_id LIMIT 3;")
    if users:
        for user in users:
            print(user)


if __name__ == "__main__":
    main()

DB HOST: musinsa-data.c07kuo6ug98z.us-east-1.rds.amazonaws.com
DB USER: admin
DB PASSWORD: qkqajrwkrnrnrn9_
DB NAME: musinsa
DB PORT: 3306
✅ DB Engine (Pool) 생성 완료
--- 사용자 조회 ---
{'product_id': 70061, 'is_crawled': 0, 'created_at': datetime.datetime(2025, 11, 27, 1, 44)}
{'product_id': 70063, 'is_crawled': 0, 'created_at': datetime.datetime(2025, 11, 27, 1, 44)}
{'product_id': 70067, 'is_crawled': 0, 'created_at': datetime.datetime(2025, 11, 27, 1, 44)}
